In [1]:
# 手搓 VAE

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar) # e^(0.5*log(var)) = sqrt(var)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))    

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [3]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [8]:
def train(epoch, model, device, train_loader, optimizer, log_interval=10):
    model.train() # model.train()会将模型设置为训练模式
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))
    
def test(epoch, model, device, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item() 
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(128, 1, 28, 28)[:n]])
                # save_image(comparison.cpu(),
                        #  'results/reconstruction_' + str(epoch) + '.png', nrow=n) 

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

    
def save_image(img, path, nrow=8):
    img = img.view(-1, 1, 28, 28) # 1 * 784 -> 1 * 28 * 28
    img = img.repeat(1, 3, 1, 1) # 1 * 28 * 28 -> 1 * 3 * 28 * 28
    img = img.detach().cpu().numpy()
    img = np.transpose(img, (0, 2, 3, 1)) # 1 * 3 * 28 * 28 -> 1 * 28 * 28 * 3
    img = np.concatenate(img, axis=1)
    plt.imsave(path, img)



In [10]:
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VAE().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3) # Adam 优化器是一种自适应学习率的优化器
    transform = transforms.Compose([transforms.ToTensor()])
    train_loader = DataLoader(datasets.MNIST('data', train=True, download=True, transform=transform), batch_size=128, shuffle=True)
    test_loader = DataLoader(datasets.MNIST('data', train=False, download=True, transform=transform), batch_size=128, shuffle=True)

    for epoch in range(1, 21):
        train(epoch, model, device, train_loader, optimizer)
        test(epoch, model, device, test_loader)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device) # 64个随机数
            sample = model.decode(sample).cpu() # 64个随机数生成的图片
            # save_image(sample, 'results/sample_' + str(epoch) + '.png')

    torch.save(model.state_dict(), 'vae.pth')
    

Train Epoch: 1 [0/60000 (0%)]	Loss: 550.419006
Train Epoch: 1 [1280/60000 (2%)]	Loss: 298.786011
Train Epoch: 1 [2560/60000 (4%)]	Loss: 238.255783
Train Epoch: 1 [3840/60000 (6%)]	Loss: 226.778244
Train Epoch: 1 [5120/60000 (9%)]	Loss: 223.877029
Train Epoch: 1 [6400/60000 (11%)]	Loss: 209.860550
Train Epoch: 1 [7680/60000 (13%)]	Loss: 204.577194
Train Epoch: 1 [8960/60000 (15%)]	Loss: 193.226303
Train Epoch: 1 [10240/60000 (17%)]	Loss: 192.714813
Train Epoch: 1 [11520/60000 (19%)]	Loss: 181.321411
Train Epoch: 1 [12800/60000 (21%)]	Loss: 179.304230
Train Epoch: 1 [14080/60000 (23%)]	Loss: 183.508591
Train Epoch: 1 [15360/60000 (26%)]	Loss: 168.406082
Train Epoch: 1 [16640/60000 (28%)]	Loss: 178.507935
Train Epoch: 1 [17920/60000 (30%)]	Loss: 167.153671
Train Epoch: 1 [19200/60000 (32%)]	Loss: 162.613266
Train Epoch: 1 [20480/60000 (34%)]	Loss: 157.970367
Train Epoch: 1 [21760/60000 (36%)]	Loss: 160.346512
Train Epoch: 1 [23040/60000 (38%)]	Loss: 159.289017
Train Epoch: 1 [24320/60000 